In [1]:
!pip install easyocr opencv-python pillow numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 14.4 MB/s eta 0:00:00


In [2]:
import cv2
import numpy as np
from PIL import Image


In [3]:
def preprocess_image(image):
    img = np.array(image.convert("RGB"))  # Convert PIL to NumPy RGB
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # Grayscale
    gray = cv2.fastNlMeansDenoising(gray, h=30)  # Denoise
    gray = cv2.equalizeHist(gray)  # Contrast boost
    _, binary = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)  # Binarize
    return img, binary


In [4]:
def run_ocr_with_boxes(image):
    result = reader.readtext(image, detail=1)
    texts = []
    for (bbox, text, conf) in result:
        if conf > 0.5:
            x_min = int(min([pt[0] for pt in bbox]))
            y_min = int(min([pt[1] for pt in bbox]))
            x_max = int(max([pt[0] for pt in bbox]))
            y_max = int(max([pt[1] for pt in bbox]))
            texts.append({
                "text": text,
                "bbox": [x_min, y_min, x_max, y_max],
                "conf": conf
            })
    return texts


In [5]:
def detect_shapes_and_arrows(binary):
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    shapes = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / float(h)
        shape_type = "arrow" if aspect_ratio > 3 or aspect_ratio < 0.3 else "box"
        shapes.append({
            "bbox": [x, y, x+w, y+h],
            "type": shape_type
        })
    return shapes


In [6]:
from google.colab import files
from PIL import Image
import numpy as np
import easyocr

# 🔍 Initialize OCR reader (use GPU if available)
reader = easyocr.Reader(['en'], gpu=True)

# 🔧 Ensure PIL format
def ensure_pil(image):
    if isinstance(image, np.ndarray):
        return Image.fromarray(image)
    return image

# 📤 Ask user to upload image
uploaded = files.upload()  # Opens file picker in Colab

# 📥 Load and convert to PIL
filename = list(uploaded.keys())[0]
image = Image.open(filename)
image = ensure_pil(image)

# 🔁 Run your pipeline
img, binary = preprocess_image(image)
texts = run_ocr_with_boxes(img)
shapes = detect_shapes_and_arrows(binary)

# ✅ You can now continue with association, graph building, and explanation


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Saving dia 03.PNG to dia 03.PNG


In [7]:
def preprocess_image(image):
    img = np.array(image.convert("RGB"))
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.fastNlMeansDenoising(gray, h=30)
    gray = cv2.equalizeHist(gray)
    _, binary = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)
    return img, binary


In [8]:
def run_ocr_with_boxes(image):
    result = reader.readtext(image, detail=1)
    texts = []
    for (bbox, text, conf) in result:
        if conf > 0.5:
            x_min = int(min([pt[0] for pt in bbox]))
            y_min = int(min([pt[1] for pt in bbox]))
            x_max = int(max([pt[0] for pt in bbox]))
            y_max = int(max([pt[1] for pt in bbox]))
            texts.append({
                "text": text,
                "bbox": [x_min, y_min, x_max, y_max],
                "conf": conf
            })
    return texts


In [9]:
def detect_shapes_and_arrows(binary):
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    shapes = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / float(h)
        shape_type = "arrow" if aspect_ratio > 3 or aspect_ratio < 0.3 else "box"
        shapes.append({
            "bbox": [x, y, x+w, y+h],
            "type": shape_type
        })
    return shapes


In [10]:
def associate_text_with_shapes(texts, shapes):
    associations = []
    for text in texts:
        tx, ty = (text["bbox"][0] + text["bbox"][2]) // 2, (text["bbox"][1] + text["bbox"][3]) // 2
        best_match = None
        min_dist = float("inf")
        for shape in shapes:
            sx, sy = (shape["bbox"][0] + shape["bbox"][2]) // 2, (shape["bbox"][1] + shape["bbox"][3]) // 2
            dist = np.sqrt((tx - sx)**2 + (ty - sy)**2)
            if dist < min_dist:
                min_dist = dist
                best_match = shape
        associations.append({
            "text": text["text"],
            "text_bbox": text["bbox"],
            "shape_bbox": best_match["bbox"],
            "shape_type": best_match["type"]
        })
    return associations


In [11]:
def build_graph(associations):
    nodes = []
    edges = []
    for i, assoc in enumerate(associations):
        nodes.append({
            "id": i,
            "label": assoc["text"],
            "bbox": assoc["text_bbox"],
            "type": assoc["shape_type"]
        })
    # Naive edge detection: arrows between boxes
    for i in range(len(nodes)):
        for j in range(len(nodes)):
            if i != j:
                if nodes[i]["type"] == "arrow":
                    edges.append({
                        "source": i,
                        "target": j,
                        "edge_type": "arrow"
                    })
    return {"nodes": nodes, "edges": edges}


In [12]:
def clean_labels(graph):
    for node in graph["nodes"]:
        label = node["label"]
        label = label.replace("\n", " ").strip()
        label = " ".join(label.split())  # remove extra spaces
        node["label"] = label
    return graph


In [13]:
def summarize_graph(graph):
    from collections import defaultdict

    # Group edges by source
    edge_map = defaultdict(list)
    for edge in graph["edges"]:
        source_label = graph["nodes"][edge["source"]]["label"]
        target_label = graph["nodes"][edge["target"]]["label"]
        edge_map[source_label].append(target_label)

    # Build flow descriptions
    flows = []
    for source, targets in edge_map.items():
        if len(targets) > 1:
            flows.append(f"{source} connects to {len(targets)} components: {', '.join(targets[:5])}{'...' if len(targets) > 5 else ''}")
        else:
            flows.append(f"{source} connects to {targets[0]}")
    return "\n".join(flows)


In [14]:
def explain_graph(graph, image_type):
    summary = summarize_graph(graph)
    node_labels = [n['label'] for n in graph['nodes']]

    prompt = (
        f"You are analyzing a {image_type.lower()} extracted from an image. "
        f"The diagram contains the following components:\n"
        f"{', '.join(node_labels)}.\n\n"
        f"Here is a summary of how they are connected:\n{summary}\n\n"
        f"Please explain the purpose and flow of this diagram. "
        f"Describe how the components interact and what the overall system might represent. "
        f"Base your explanation only on the extracted elements. Avoid assumptions or external knowledge."
    )

    response = generator(prompt, do_sample=False)
    return response[0]['generated_text']



In [15]:
!pip install -U transformers accelerate huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 44.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.35.0
    Uninstalling huggingface-hub-0.35.0:
      Successfully uninstalled huggingface-hub-0.35.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [16]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get("HF_TOKEN")
login(hf_token)


In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id
)


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Device set to use cuda:0


In [18]:
import ipywidgets as widgets
from IPython.display import display

type_selector = widgets.Dropdown(
    options=["Diagram", "Table"],
    value="Diagram",
    description="Image Type:",
    style={'description_width': 'initial'}
)
display(type_selector)

# Then use: type_selector.value instead of hardcoded "Diagram"


Dropdown(description='Image Type:', options=('Diagram', 'Table'), style=DescriptionStyle(description_width='in…

In [20]:
# 🔁 Continue pipeline
associations = associate_text_with_shapes(texts, shapes)
graph = build_graph(associations)
graph = clean_labels(graph)



# 💬 Generate explanation using Gemma 2B
explanation = explain_graph(graph, type_selector.value)


# 📜 Display result
print("🧠 Explanation:\n")
print(explanation)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🧠 Explanation:

You are analyzing a diagram extracted from an image. The diagram contains the following components:
BR, CLK, Baudrate, Divisor, WISHBONE, RxD PAD [, WISFBONE, SIGNALS, 904, Transmitter, ~TxD_PAD_, IntTx 0, IntRx 0, Activate.

Here is a summary of how they are connected:
RxD PAD [ connects to 13 components: BR, CLK, Baudrate, Divisor, WISHBONE...
~TxD_PAD_ connects to 13 components: BR, CLK, Baudrate, Divisor, WISHBONE...
IntTx 0 connects to 13 components: BR, CLK, Baudrate, Divisor, WISHBONE...
IntRx 0 connects to 13 components: BR, CLK, Baudrate, Divisor, WISHBONE...

Please explain the purpose and flow of this diagram. Describe how the components interact and what the overall system might represent. Base your explanation only on the extracted elements. Avoid assumptions or external knowledge.

**Purpose:**

The purpose of this diagram is to provide a high-level overview of a communication system. It shows how different components interact with each other to facilitate

In [ ]:
import gradio as gr

def process_image(image, image_type):
    image = ensure_pil(image)
    img, binary = preprocess_image(image)
    texts = run_ocr_with_boxes(img)
    shapes = detect_shapes_and_arrows(binary)
    associations = associate_text_with_shapes(texts, shapes)
    graph = build_graph(associations)
    graph = clean_labels(graph)
    explanation = explain_graph(graph, image_type)  # image_type will come from Radio
    return explanation

# 🖼️ Build Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Diagram/Table Explainer with Gemma 2B")

    with gr.Row():
        image_input = gr.Image(type="pil", label="Upload Diagram or Table")
        image_type_selector = gr.Radio(["Diagram", "Table"], value="Diagram", label="Image Type")

    with gr.Row():
        submit_btn = gr.Button("🔍 Extract & Explain")

    explanation_output = gr.Textbox(label="Explanation", lines=20, interactive=False, show_copy_button=True)

    submit_btn.click(fn=process_image, inputs=[image_input, image_type_selector], outputs=explanation_output)

demo.launch(debug=True)



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0eea57b8cf33c1fa7d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
